<a href="https://colab.research.google.com/github/ClaudeCoulombe/VIARENA/blob/master/Labos/Lab-CIFAR_10/Identification_Objets-ResConv-VariantesArchitectures-CIFAR_10.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Rappel - Fonctionnement d'un carnet web iPython

* Pour exécuter le code contenu dans une cellule d'un carnet iPython, cliquez dans la cellule et faites (⇧↵, shift-enter) 
* Le code d'un carnet iPython s'exécute séquentiellement de haut en bas de la page. Souvent, l'importation d'une bibliothèque Python ou l'initialisation d'une variable est préalable à l'exécution d'une cellule située plus bas. Il est donc recommandé d'exécuter les cellules en séquence. Enfin, méfiez-vous des retours en arrière qui peuvent réinitialiser certaines variables.

# Identification d'objets à partir de photos - jeu de données CIFAR-10
## Labo réseau convolutif - variantes d'architecture

Inspirations: 

* TensorFlow Tutorial - Google: <a href="https://www.tensorflow.org/tutorials/images/cnn?hl=fr" target='_blank'>Tutoriel TensorFlow - Réseau neuronal convolutif</a>
* Machine Learning Mastery - Jason Brownlee: <a href="https://machinelearningmastery.com/how-to-develop-a-cnn-from-scratch-for-cifar-10-photo-classification/" target='_blank'>How to Develop a CNN From Scratch for CIFAR-10 Photo Classification</a> 


## Jeu de données - photos CIFAR-10
L'ensemble de données CIFAR-10 (Canadian Institute For Advanced Research) comporte 60 000 photographies en couleur de 32×32 pixels d'objets de 10 classes différentes. Il est relativement simple d'atteindre une précision de 80 %. On peut obtenir des performances de 90 % avec ces données avec des réseaux neuronaux convolutifs. 

* 0 : avion
* 1 : automobile
* 2 : oiseau
* 3 : chat
* 4 : cerf
* 5 : chien
* 6 : grenouille
* 7 : cheval
* 8 : bateau
* 9 : camion


In [ ]:
import os
import matplotlib.pyplot as plt

import tensorflow as tf
print("TensorFlow version:",tf.__version__)
import keras
print("Keras version:",keras.__version__)

# Importer le jeu de données CIFAR-10
from keras.datasets import cifar10

dic_noms_classe = { 
    0 : "avion",
    1 : "automobile",
    2 : "oiseau",
    3 : "chat",
    4 : "cerf",
    5 : "chien",
    6 : "grenouille",
    7 : "cheval",
    8 : "bateau",
    9 : "camion",
}

# Lire le jeu de données CIFAR-10 et le diviser entre
# les données d'entrainement et les données de test
(attributs_entrainement, classes_cibles_entrainement), (attributs_test, classes_cibles_test) = cifar10.load_data()

# résumé des données 
print()
print('Entraînement: attributs=%s, classes-cibles=%s' % (attributs_entrainement.shape, classes_cibles_entrainement.shape))
print('Test: attributs=%s, classes-cibles=%s' % (attributs_test.shape, classes_cibles_test.shape))

# Afficher les 24 premières images
print()
print("Quelques images avec leur étiquette de classe-cible...")
%matplotlib inline
# définir lagrill d'affichage des images
fig, axes = plt.subplots(nrows=4,ncols=6,figsize=(10,8))
for i_rangee in range(0,4):
    for i_colonne in range(0,6):
        axes[i_rangee,i_colonne].set_title(dic_noms_classe[int(classes_cibles_entrainement[i_rangee*6+i_colonne])],
                                           fontsize=10)
        axes[i_rangee,i_colonne].imshow(attributs_entrainement[i_rangee*6+i_colonne])
plt.show()

## Prétraitement des données

In [ ]:
# Conversion des classes-cibles en vecteurs binaires à un bit discriminant
from tensorflow.keras.utils import to_categorical

classes_cibles_entrainement = to_categorical(classes_cibles_entrainement)
classes_cibles_test = to_categorical(classes_cibles_test)

# Normalisation

def normalisation(entrainement, test):
    # convertir de nombres entiers à nombres décimaux
    entrainement_normalise = entrainement.astype('float32')
    test_normalise = test.astype('float32')
    # normalisation à un nombre entre 0 et 1
    entrainement_normalise = entrainement_normalise / 255.0
    test_normalise = test_normalise / 255.0
    return entrainement_normalise, test_normalise

attributs_entrainement, attributs_test = normalisation(attributs_entrainement, attributs_test)

print("Normalisation terminée!")

# 1- Modèle de base

In [ ]:
## Construction du modèle

from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

print("Création d'un modèle de base...")

input_shape = (32, 32, 3)
batch_size = 32
# ou nombre de classes
nombre_classes_cibles = 10

modele_de_base = Sequential()
# Apprentissage et extraction des attributs
modele_de_base.add(Conv2D(32, 
                          kernel_size=(3,3),
                          activation='relu',
                          kernel_initializer='glorot_uniform',
                          padding='same',
                          strides=(1,1),
                          input_shape=input_shape))
modele_de_base.add(MaxPooling2D(pool_size=(2,2)))

# Classification des images
# Classificateur perceptron multicouche
modele_de_base.add(Flatten())
modele_de_base.add(Dense(128,
                         activation='relu',
                         kernel_initializer='glorot_uniform'))
modele_de_base.add(Dense(nombre_classes_cibles, 
                         activation='softmax'))

print()
print("Description du modèle de base:")
modele_de_base.summary()

In [ ]:
# Compilation du modèle

print()
print("Compilation du modèle de base...")

modele_de_base.compile(loss="categorical_crossentropy", 
                       optimizer="adam", 
                       metrics=["accuracy"])

## Entraînement du modèle de base

In [ ]:
# Entraînement du modèle

print()
print("Entraînement du modèle de base...")

batch_size = 128
epochs = 15

traces_entrainement = modele_de_base.fit(attributs_entrainement,
                                         classes_cibles_entrainement,
                                         batch_size=batch_size,
                                         epochs=epochs,
                                         validation_split=0.1)

## Évaluation du modèle de base

In [ ]:
# Évaluation du modèle
print()
print("Évaluation du modèle de base...")

resultats = modele_de_base.evaluate(attributs_test, classes_cibles_test, verbose=0)
print("Exactitude test: {:.2f}%".format(resultats[1]*100))

## Affichage courbes d'entraînement du modèle de base

In [ ]:
# Affichage des courbes d'entraînement
hauteur = 8
plt.subplots(figsize=(hauteur,1.618*hauteur))
plt.subplot(211)
plt.title('Erreur entropie croisée - modele_de_base')
plt.plot(traces_entrainement.history['loss'], color='blue', label='courbe entraînement')
plt.plot(traces_entrainement.history['val_loss'], color='orange', label='courbe test')
plt.ylabel("Erreur")
plt.xlabel("Nombre d'époques")
plt.legend()
plt.show()
# tracer l'exactitude
plt.subplots(figsize=(hauteur,1.618*hauteur))
plt.subplot(212)
plt.title('\nExactitude de la classification - modele_de_base')
plt.plot(traces_entrainement.history['accuracy'], color='blue', label='courbe entraînement')
plt.plot(traces_entrainement.history['val_accuracy'], color='orange', label='courbe test')
plt.ylabel("Exactitude")
plt.xlabel("Nombre d'époques")
plt.legend()
plt.show()
# Sauvegarde du graphique en format .png
# nom_graphique = "modele_de_base-courbes_entraînement.png"
# plt.savefig(nom_graphique)
# plt.close()

# 2- Modèle 2 fois plus profond

In [ ]:
# Construction du modèle

from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

print("Création d'un modèle 2 fois plus profond...")

input_shape = (32, 32, 3)
batch_size = 32
# ou nombre de classes
nombre_classes_cibles = 10

modele_2X_plus_profond = Sequential()
# Apprentissage et extraction des attributs
modele_2X_plus_profond.add(Conv2D(32, 
                         kernel_size=(3,3), 
                         activation='relu', 
                         kernel_initializer='glorot_uniform',
                         padding='same',
                         strides=(1,1),
                         input_shape=input_shape))
modele_2X_plus_profond.add(MaxPooling2D(pool_size=(2,2)))
modele_2X_plus_profond.add(Conv2D(64,
                          kernel_size=(3,3), 
                          activation='relu', 
                          kernel_initializer='glorot_uniform',
                          padding='same',
                          strides=(1,1),))
modele_2X_plus_profond.add(MaxPooling2D(pool_size=(2,2)))

# Classification des images
modele_2X_plus_profond.add(Flatten())
modele_2X_plus_profond.add(Dense(128, 
                         activation='relu', 
                         kernel_initializer='glorot_uniform'))
modele_2X_plus_profond.add(Dense(nombre_classes_cibles, 
                         activation='softmax'))
print()
print("Description du modèle 2 fois plus profond:")
modele_2X_plus_profond.summary()

In [ ]:
# Compilation du modèle

print()
print("Compilation du modèle 2 fois plus profond...")

modele_2X_plus_profond.compile(loss="categorical_crossentropy", 
                       optimizer="adam", 
                       metrics=["accuracy"])

## Entraînement modèle 2 fois plus profond

In [ ]:
# Entraînement du modèle

print()
print("Entraînement du modèle 2 fois plus profond...")

batch_size = 128
epochs = 15

traces_entrainement = modele_2X_plus_profond.fit(attributs_entrainement,
                                                 classes_cibles_entrainement,
                                                 batch_size=batch_size,
                                                 epochs=epochs,
                                                 validation_split=0.1)

## Évaluation modèle 2 fois plus profond

In [ ]:
# Évaluation du modèle

print()
print("Évaluation du modèle 2 fois plus profond...")

resultats = modele_2X_plus_profond.evaluate(attributs_test, classes_cibles_test, verbose=0)
print("Exactitude test: {:.2f}%".format(resultats[1]*100))

## Affichage courbes d'entraînement, modèle 2 fois plus profond

In [ ]:
# Affichage des courbes d'entraînement
hauteur = 8
plt.subplots(figsize=(hauteur,1.618*hauteur))
plt.subplot(211)
plt.title('Erreur entropie croisée - modele_2X_plus_profond')
plt.plot(traces_entrainement.history['loss'], color='blue', label='courbe entraînement')
plt.plot(traces_entrainement.history['val_loss'], color='orange', label='courbe test')
plt.ylabel("Erreur")
plt.xlabel("Nombre d'époques")
plt.legend()
plt.show()
# tracer l'exactitude
plt.subplots(figsize=(hauteur,1.618*hauteur))
plt.subplot(212)
plt.title('\nExactitude de la classification - modele_2X_plus_profond')
plt.plot(traces_entrainement.history['accuracy'], color='blue', label='courbe entraînement')
plt.plot(traces_entrainement.history['val_accuracy'], color='orange', label='courbe test')
plt.ylabel("Exactitude")
plt.xlabel("Nombre d'époques")
plt.legend()
plt.show()
# Sauvegarde du graphique en format .png
# nom_graphique = "modele_2X_plus_profond-courbes_entraînement.png"
# plt.savefig(nom_graphique)
# plt.close()

# 3- Modèle 3 fois plus profond

In [ ]:
# Construction du modèle

from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

print("Création du modèle 3 fois plus profond...")

input_shape = (32, 32, 3)
batch_size = 32
# ou nombre de classes
nombre_classes_cibles = 10

modele_3X_plus_profond = Sequential()
# Apprentissage et extraction des attributs
modele_3X_plus_profond.add(Conv2D(32, 
                         kernel_size=(3,3), 
                         activation='relu', 
                         kernel_initializer='glorot_uniform',
                         padding='same',
                         strides=(1,1),
                         input_shape=input_shape))
modele_3X_plus_profond.add(MaxPooling2D(pool_size=(2,2)))

modele_3X_plus_profond.add(Conv2D(64,
                          kernel_size=(3,3), 
                          activation='relu', 
                          kernel_initializer='glorot_uniform',
                          padding='same',
                          strides=(1,1),))
modele_3X_plus_profond.add(MaxPooling2D(pool_size=(2,2)))

modele_3X_plus_profond.add(Conv2D(128,
                          kernel_size=(3,3), 
                          activation='relu', 
                          kernel_initializer='glorot_uniform',
                          padding='same',
                          strides=(1,1),))
modele_3X_plus_profond.add(MaxPooling2D(pool_size=(2,2)))

# Classification des images
modele_3X_plus_profond.add(Flatten())
modele_3X_plus_profond.add(Dense(128,
                           activation='relu',
                           kernel_initializer='glorot_uniform'))
modele_3X_plus_profond.add(Dense(nombre_classes_cibles, 
                         activation='softmax'))
print()
print("Description du modèle 3 fois plus profond:")
modele_3X_plus_profond.summary()

In [ ]:
# Compilation du modèle

print()
print("Compilation du modèle 3 fois plus profond...")

modele_3X_plus_profond.compile(loss="categorical_crossentropy", 
                       optimizer="adam", 
                       metrics=["accuracy"])

## Entraînement modèle 3 fois plus profond

In [ ]:
# Entraînement du modèle

print()
print("Entraînement du modèle 3 fois plus profond...")

batch_size = 128
epochs = 15

traces_entrainement = modele_3X_plus_profond.fit(attributs_entrainement,
                                                 classes_cibles_entrainement,
                                                 batch_size=batch_size,
                                                 epochs=epochs,
                                                 validation_split=0.1)

## Évaluation modèle 3 fois plus profond

In [ ]:
# Évaluation du modèle

print()
print("Évaluation du modèle 3 fois plus profond...")

resultats = modele_3X_plus_profond.evaluate(attributs_test, classes_cibles_test, verbose=0)
print("Exactitude test: {:.2f}%".format(resultats[1]*100))

## Affichage courbes d'entraînement, modèle 3 fois plus profond

In [ ]:
# Affichage des courbes d'entraînement
hauteur = 8
plt.subplots(figsize=(hauteur,1.618*hauteur))
plt.subplot(211)
plt.title('Erreur entropie croisée  - modele_3X_plus_profond')
plt.plot(traces_entrainement.history['loss'], color='blue', label='courbe entraînement')
plt.plot(traces_entrainement.history['val_loss'], color='orange', label='courbe test')
plt.ylabel("Erreur")
plt.xlabel("Nombre d'époques")
plt.legend()
plt.show()
# tracer l'exactitude
plt.subplots(figsize=(hauteur,1.618*hauteur))
plt.subplot(212)
plt.title('\nExactitude de la classification - modele_3X_plus_profond')
plt.plot(traces_entrainement.history['accuracy'], color='blue', label='courbe entraînement')
plt.plot(traces_entrainement.history['val_accuracy'], color='orange', label='courbe test')
plt.ylabel("Exactitude")
plt.xlabel("Nombre d'époques")
plt.legend()
plt.show()
# Sauvegarde du graphique en format .png
# nom_graphique = "modele_3X_plus_profond-courbes_entraînement.png"
# plt.savefig(nom_graphique)
# plt.close()

# 4- Modèle 4 fois plus profond

In [ ]:
# Construction du modèle

from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

print("Création du modèle 4 fois plus profond...")

input_shape = (32, 32, 3)
batch_size = 32
# ou nombre de classes
nombre_classes_cibles = 10

modele_4X_plus_profond = Sequential()
# Apprentissage et extraction des attributs
modele_4X_plus_profond.add(Conv2D(32, 
                         kernel_size=(3,3), 
                         activation='relu', 
                         kernel_initializer='glorot_uniform',
                         padding='same',
                         strides=(1,1),
                         input_shape=input_shape))
modele_4X_plus_profond.add(MaxPooling2D(pool_size=(2,2)))

modele_4X_plus_profond.add(Conv2D(64,
                          kernel_size=(3,3), 
                          activation='relu', 
                          kernel_initializer='glorot_uniform',
                          padding='same',
                          strides=(1,1),))
modele_4X_plus_profond.add(MaxPooling2D(pool_size=(2,2)))

modele_4X_plus_profond.add(Conv2D(128,
                          kernel_size=(3,3), 
                          activation='relu', 
                          kernel_initializer='glorot_uniform',
                          padding='same',
                          strides=(1,1),))
modele_4X_plus_profond.add(MaxPooling2D(pool_size=(2,2)))

modele_4X_plus_profond.add(Conv2D(256,
                          kernel_size=(3,3), 
                          activation='relu', 
                          kernel_initializer='glorot_uniform',
                          padding='same',
                          strides=(1,1),))
modele_4X_plus_profond.add(MaxPooling2D(pool_size=(2,2)))

# Classification des images
modele_4X_plus_profond.add(Flatten())
modele_4X_plus_profond.add(Dense(128, 
                         activation='relu', 
                         kernel_initializer='glorot_uniform'))
modele_4X_plus_profond.add(Dense(nombre_classes_cibles, 
                         activation='softmax'))

print()
print("Description du modèle 4 fois plus profond:")
modele_4X_plus_profond.summary()

In [ ]:
# Compilation du modèle

print()
print("Compilation du modèle 4 fois plus profond...")

modele_4X_plus_profond.compile(loss="categorical_crossentropy", 
                       optimizer="adam", 
                       metrics=["accuracy"])

## Entraînement modèle 4 fois plus profond

In [ ]:
# Entraînement du modèle

print()
print("Entraînement du modèle 4 fois plus profond...")

batch_size = 128
epochs = 15

traces_entrainement = modele_4X_plus_profond.fit(attributs_entrainement,
                                                 classes_cibles_entrainement,
                                                 batch_size=batch_size,
                                                 epochs=epochs,
                                                 validation_split=0.1)

## Évaluation modèle 4 fois plus profond

In [ ]:
# Évaluation du modèle

print()
print("Évaluation du modèle 4 fois plus profond...")

resultats = modele_4X_plus_profond.evaluate(attributs_test, classes_cibles_test, verbose=0)
print("Exactitude test: {:.2f}%".format(resultats[1]*100))

## Affichage courbes d'entraînement, modèle 4 fois plus profond

In [ ]:
# Affichage des courbes d'entraînement
hauteur = 8
plt.subplots(figsize=(hauteur,1.618*hauteur))
plt.subplot(211)
plt.title('Erreur entropie croisée - modele_4X_plus_profond')
plt.plot(traces_entrainement.history['loss'], color='blue', label='courbe entraînement')
plt.plot(traces_entrainement.history['val_loss'], color='orange', label='courbe test')
plt.ylabel("Erreur")
plt.xlabel("Nombre d'époques")
plt.legend()
plt.show()
# tracer l'exactitude
plt.subplots(figsize=(hauteur,1.618*hauteur))
plt.subplot(212)
plt.title('\nExactitude de la classification - modele_4X_plus_profond')
plt.plot(traces_entrainement.history['accuracy'], color='blue', label='courbe entraînement')
plt.plot(traces_entrainement.history['val_accuracy'], color='orange', label='courbe test')
plt.ylabel("Exactitude")
plt.xlabel("Nombre d'époques")
plt.legend()
plt.show()
# Sauvegarde du graphique en format .png
# nom_graphique = "modele_4X_plus_profond-courbes_entraînement.png"
# plt.savefig(nom_graphique)
# plt.close()

# 5- Modèle 3 fois plus profond régularisé par extinction

In [ ]:
# Construction d'un modèle de base

from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout

print("Création du modèle 3 fois plus profond régularisé...")

input_shape = (32, 32, 3)
batch_size = 32
# ou nombre de classes
nombre_classes_cibles = 10

modele_3X_plus_profond_regularise = Sequential()

# Apprentissage et extraction des attributs
modele_3X_plus_profond_regularise.add(Conv2D(32, 
                         kernel_size=(3,3), 
                         activation='relu', 
                         kernel_initializer='glorot_uniform',
                         padding='same',
                         strides=(1,1),
                         input_shape=input_shape))
modele_3X_plus_profond_regularise.add(MaxPooling2D(pool_size=(2,2)))
# Régularisation par extinction de neurones (dropout) 
modele_3X_plus_profond_regularise.add(Dropout(0.2))

modele_3X_plus_profond_regularise.add(Conv2D(64,
                          kernel_size=(3,3), 
                          activation='relu', 
                          kernel_initializer='glorot_uniform',
                          padding='same',
                          strides=(1,1),))
modele_3X_plus_profond_regularise.add(MaxPooling2D(pool_size=(2,2)))
# Régularisation par extinction de neurones (dropout) 
modele_3X_plus_profond_regularise.add(Dropout(0.2))

modele_3X_plus_profond_regularise.add(Conv2D(128,
                          kernel_size=(3,3), 
                          activation='relu', 
                          kernel_initializer='glorot_uniform',
                          padding='same',
                          strides=(1,1),))
modele_3X_plus_profond_regularise.add(MaxPooling2D(pool_size=(2,2)))
# Régularisation par extinction de neurones (dropout) 
modele_3X_plus_profond_regularise.add(Dropout(0.2))

# Classification des images
modele_3X_plus_profond_regularise.add(Flatten())
modele_3X_plus_profond_regularise.add(Dense(128, 
                         activation='relu', 
                         kernel_initializer='glorot_uniform'))
modele_3X_plus_profond_regularise.add(Dense(nombre_classes_cibles, 
                         activation='softmax'))

print()
print("Description du modèle 3 fois plus profond régularisé:")
modele_3X_plus_profond_regularise.summary()

In [ ]:
# Compilation du modèle

print()
print("Compilation du modèle 4 fois plus profond...")

modele_3X_plus_profond_regularise.compile(loss="categorical_crossentropy",
                                          optimizer="adam",
                                          metrics=["accuracy"])

## Entraînement modèle 3 fois plus profond régularisé par extinction

In [ ]:
# Entraînement du modèle

print()
print("Entraînement du modèle 3 fois plus profond régularisé...")

batch_size = 128
epochs = 15

traces_entrainement = modele_3X_plus_profond_regularise.fit(attributs_entrainement,
                                                            classes_cibles_entrainement,
                                                            batch_size=batch_size,
                                                            epochs=epochs,
                                                            validation_split=0.1)

## Évaluation modèle 3 fois plus profond régularisé par extinction

In [ ]:
# Évaluation du modèle
print()
print("Évaluation du modèle 3 fois plus profond régularisé...")

resultats = modele_3X_plus_profond_regularise.evaluate(attributs_test, classes_cibles_test, verbose=0)
print("Exactitude test: {:.2f}%".format(resultats[1]*100))

## Affichage courbes d'entraînement, modèle 3 fois plus profond régularisé par extinction 

In [ ]:
# Affichage des courbes d'entraînement
hauteur = 8
plt.subplots(figsize=(hauteur,1.618*hauteur))
plt.subplot(211)
plt.title('Erreur entropie croisée - modele_3X_plus_profond_regularise')
plt.plot(traces_entrainement.history['loss'], color='blue', label='courbe entraînement')
plt.plot(traces_entrainement.history['val_loss'], color='orange', label='courbe test')
plt.ylabel("Erreur")
plt.xlabel("Nombre d'époques")
plt.legend()
plt.show()
# tracer l'exactitude
plt.subplots(figsize=(hauteur,1.618*hauteur))
plt.subplot(212)
plt.title('\nExactitude de la classification - modele_3X_plus_profond_regularise')
plt.plot(traces_entrainement.history['accuracy'], color='blue', label='courbe entraînement')
plt.plot(traces_entrainement.history['val_accuracy'], color='orange', label='courbe test')
plt.ylabel("Exactitude")
plt.xlabel("Nombre d'époques")
plt.legend()
plt.show()
# Sauvegarde du graphique en format .png
# nom_graphique = "modele_3X_plus_profond_regularise-courbes_entraînement.png"
# plt.savefig(nom_graphique)
# plt.close()

# 6- Modèle 3 fois plus profond régularisé (L2 + extinction)


In [ ]:
# Construction d'un modèle de base

from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras import regularizers

print("Création du modèle 3 fois plus profond régularisé (L2 + extinction)...")

input_shape = (32, 32, 3)
batch_size = 32
# ou nombre de classes
nombre_classes_cibles = 10
diminution_poids = 0.01

modele_3Xplus_profond_regL2Xtn = Sequential()

# Apprentissage et extraction des attributs
modele_3Xplus_profond_regL2Xtn.add(Conv2D(32, 
                         kernel_size=(3,3), 
                         activation='elu', 
                         kernel_initializer='glorot_uniform',
                         kernel_regularizer=regularizers.l2(diminution_poids),
                         padding='same',
                         strides=(1,1),
                         input_shape=input_shape))
modele_3Xplus_profond_regL2Xtn.add(MaxPooling2D(pool_size=(2,2)))
# Régularisation par extinction de neurones (dropout) 
modele_3Xplus_profond_regL2Xtn.add(Dropout(0.2))

modele_3Xplus_profond_regL2Xtn.add(Conv2D(64,
                          kernel_size=(3,3), 
                          activation='elu', 
                          kernel_initializer='glorot_uniform',
                          kernel_regularizer=regularizers.l2(diminution_poids),
                          padding='same',
                          strides=(1,1),))
modele_3Xplus_profond_regL2Xtn.add(MaxPooling2D(pool_size=(2,2)))
# Régularisation par extinction de neurones (dropout) 
modele_3Xplus_profond_regL2Xtn.add(Dropout(0.2))

modele_3Xplus_profond_regL2Xtn.add(Conv2D(128,
                          kernel_size=(3,3), 
                          activation='elu', 
                          kernel_initializer='glorot_uniform',
                          kernel_regularizer=regularizers.l2(diminution_poids),
                          padding='same',
                          strides=(1,1),))
modele_3Xplus_profond_regL2Xtn.add(MaxPooling2D(pool_size=(2,2)))
# Régularisation par extinction de neurones (dropout) 
modele_3Xplus_profond_regL2Xtn.add(Dropout(0.2))

# Classification des images
modele_3Xplus_profond_regL2Xtn.add(Flatten())
modele_3Xplus_profond_regL2Xtn.add(Dense(128, 
                         activation='elu', 
                         kernel_initializer='glorot_uniform'))
modele_3Xplus_profond_regL2Xtn.add(Dense(nombre_classes_cibles, 
                         activation='softmax'))

print()
print("Description du modèle 3 fois plus profond régularisé (L2 + extinction):")
modele_3Xplus_profond_regL2Xtn.summary()

In [ ]:
# Compilation du modèle

print()
print("Compilation du modèle 3 fois plus profond régularisé (L2 + extinction)...")

modele_3Xplus_profond_regL2Xtn.compile(loss="categorical_crossentropy", 
                       optimizer="adam", 
                       metrics=["accuracy"])

## Entraînement modèle 3 fois plus profond régularisé (L2 + extinction)

In [ ]:
# Entraînement du modèle

print()
print("Entraînement du modèle 3 fois plus profond régularisé (L2 + extinction)...")

batch_size = 128
epochs = 15

traces_entrainement = modele_3Xplus_profond_regL2Xtn.fit(attributs_entrainement,
                                                         classes_cibles_entrainement,
                                                         batch_size=batch_size,
                                                         epochs=epochs,
                                                         validation_split=0.1)

## Évaluation modèle 3 fois plus profond régularisé (L2 + extinction)

In [ ]:
# Évaluation du modèle
print()
print("Évaluation du modèle 3 fois plus profond régularisé (L2 + extinction)...")

resultats = modele_3Xplus_profond_regL2Xtn.evaluate(attributs_test, classes_cibles_test, verbose=0)
print("Exactitude test: {:.2f}%".format(resultats[1]*100))

## Affichage courbes d'entraînement, modèle 3 fois plus profond régularisé (L2 + extinction) 

In [ ]:
# Affichage des courbes d'entraînement
hauteur = 8
plt.subplots(figsize=(hauteur,1.618*hauteur))
plt.subplot(211)
plt.title('Erreur entropie croisée - modele_3Xplus_profond_regL2Xtn')
plt.plot(traces_entrainement.history['loss'], color='blue', label='courbe entraînement')
plt.plot(traces_entrainement.history['val_loss'], color='orange', label='courbe test')
plt.ylabel("Erreur")
plt.xlabel("Nombre d'époques")
plt.legend()
plt.show()
# tracer l'exactitude
plt.subplots(figsize=(hauteur,1.618*hauteur))
plt.subplot(212)
plt.title('\nExactitude de la classification - modele_3Xplus_profond_regL2Xtn')
plt.plot(traces_entrainement.history['accuracy'], color='blue', label='courbe entraînement')
plt.plot(traces_entrainement.history['val_accuracy'], color='orange', label='courbe test')
plt.ylabel("Exactitude")
plt.xlabel("Nombre d'époques")
plt.legend()
plt.show()
# Sauvegarde du graphique en format .png
# nom_graphique = "modele_3Xplus_profond_regL2Xtn-courbes_entraînement.png"
# plt.savefig(nom_graphique)
# plt.close()

# 7- Modèle 3 fois plus profond régularisé et normalisation des lots


In [ ]:
# Construction d'un modèle de base

from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization

print("Création du modèle 3 fois plus profond régularisé et normalisation des lots...")

input_shape = (32, 32, 3)
batch_size = 32
# ou nombre de classes
nombre_classes_cibles = 10

modele_3X_plus_profond_reg_norm = Sequential()

# Apprentissage et extraction des attributs
modele_3X_plus_profond_reg_norm.add(Conv2D(32, 
                         kernel_size=(3,3), 
                         activation='elu', 
                         kernel_initializer='glorot_uniform',
                         padding='same',
                         strides=(1,1),
                         input_shape=input_shape))
modele_3X_plus_profond_reg_norm.add(BatchNormalization())
modele_3X_plus_profond_reg_norm.add(MaxPooling2D(pool_size=(2,2)))
# Régularisation par extinction de neurones (dropout) 
modele_3X_plus_profond_reg_norm.add(Dropout(0.2))

modele_3X_plus_profond_reg_norm.add(Conv2D(64,
                          kernel_size=(3,3), 
                          activation='elu', 
                          kernel_initializer='glorot_uniform',
                          padding='same',
                          strides=(1,1),))
modele_3X_plus_profond_reg_norm.add(BatchNormalization())
modele_3X_plus_profond_reg_norm.add(MaxPooling2D(pool_size=(2,2)))
# Régularisation par extinction de neurones (dropout) 
modele_3X_plus_profond_reg_norm.add(Dropout(0.2))

modele_3X_plus_profond_reg_norm.add(Conv2D(128,
                          kernel_size=(3,3), 
                          activation='elu', 
                          kernel_initializer='glorot_uniform',
                          padding='same',
                          strides=(1,1),))
modele_3X_plus_profond_reg_norm.add(BatchNormalization())
modele_3X_plus_profond_reg_norm.add(MaxPooling2D(pool_size=(2,2)))
# Régularisation par extinction de neurones (dropout) 
modele_3X_plus_profond_reg_norm.add(Dropout(0.2))

# Classification des images
modele_3X_plus_profond_reg_norm.add(Flatten())
modele_3X_plus_profond_reg_norm.add(Dense(128, 
                         activation='elu', 
                         kernel_initializer='glorot_uniform'))
modele_3X_plus_profond_reg_norm.add(Dense(nombre_classes_cibles, 
                         activation='softmax'))

print()
print("Description du modèle 3 fois plus profond régularisé et normalisation des lots :")
modele_3X_plus_profond_reg_norm.summary()

In [ ]:
# Compilation du modèle

print()
print("Compilation du modèle 3 fois plus profond régularisé et normalisation des lots...")

modele_3X_plus_profond_reg_norm.compile(loss="categorical_crossentropy", 
                       optimizer="adam", 
                       metrics=["accuracy"])

## Entraînement modèle 3 fois plus profond régularisé et normalisation des lots

In [ ]:
# Entraînement du modèle

print()
print("Entraînement du modèle 3 fois plus profond régularisé et normalisation des lots...")

batch_size = 128
epochs = 15

traces_entrainement = modele_3X_plus_profond_reg_norm.fit(attributs_entrainement,
                                                          classes_cibles_entrainement,
                                                          batch_size=batch_size,
                                                          epochs=epochs,
                                                          validation_split=0.1)

## Évaluation modèle 3 fois plus profond régularisé et normalisation des lots

In [ ]:
# Évaluation du modèle
print()
print("Évaluation du modèle 3 fois plus profond régularisé et normalisation des lots...")

resultats = modele_3X_plus_profond_reg_norm.evaluate(attributs_test, classes_cibles_test, verbose=0)
print("Exactitude test: {:.2f}%".format(resultats[1]*100))

## Affichage courbes d'entraînement, modèle 3X profond régularisé & normalisation lots 

In [ ]:
# Affichage des courbes d'entraînement
hauteur = 8
plt.subplots(figsize=(hauteur,1.618*hauteur))
plt.subplot(211)
plt.title('Erreur entropie croisée - modèle 3X profond régularisé & normalisation lots')
plt.plot(traces_entrainement.history['loss'], color='blue', label='courbe entraînement')
plt.plot(traces_entrainement.history['val_loss'], color='orange', label='courbe test')
plt.ylabel("Erreur")
plt.xlabel("Nombre d'époques")
plt.legend()
plt.show()
# tracer l'exactitude
plt.subplots(figsize=(hauteur,1.618*hauteur))
plt.subplot(212)
plt.title('\nExactitude de la classification - modèle 3X profond régularisé & normalisation lots')
plt.plot(traces_entrainement.history['accuracy'], color='blue', label='courbe entraînement')
plt.plot(traces_entrainement.history['val_accuracy'], color='orange', label='courbe test')
plt.ylabel("Exactitude")
plt.xlabel("Nombre d'époques")
plt.legend()
plt.show()
# Sauvegarde du graphique en format .png
# nom_graphique = "modele_3Xplus_profond_regularise_&_normalisation_lots-courbes_entraînement.png"
# plt.savefig(nom_graphique)
# plt.close()

In [ ]:
print("Carnet IPython exécution terminée.")